## 작업형3

### 문제1. 주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/clam.csv")

#패널티는 부과하지 않는다 = 일반적인 로지스틱 회귀모델을 사용하라는 것

#데이터 셋 분할
df.head()
train = df.iloc[:210]
test = df.iloc[210:]
print(train.shape, test.shape)




(210, 6) (90, 6)


In [ ]:
from statsmodels.formula.api import logit
import numpy as np

#1) 로지스틱 회귀 모델 생성 및 학습
model = logit("gender ~ weight", data=train).fit()  # 로지스틱회귀("종속변수" ~ "독립변수", 데이터는 트레인데이터쓰자). 학습시키자
print(model.summary())

#2) 오즈비 계산
print(np.exp(0.0047)) #이건 weight의 coef 값을 넣어서 어림으로 계산한거고
print(np.exp(model.params['weight']))  #이건 위에처럼 어림잡지않고, params로 소수점 끝까지 정확히 계산한것임


Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Thu, 26 Dec 2024   Pseudo R-squ.:                0.003431
Time:                        09:26:58   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.

오즈비(Odds Ratio) 계산에 대해 설명해드리겠습니다.
로지스틱 회귀에서 계수(coefficient)를 해석할 때 np.exp()를 사용하여 오즈비를 계산하는 이유는:

1. 로지스틱 회귀의 특성:
로지스틱 회귀는 log(odds)를 모델링합니다
계수는 log(odds)의 변화량을 나타냅니다
이 값을 직접적으로 해석하기는 어렵습니다

2.지수변환(exp)의 의미:
np.exp(coefficient)를 하면 log scale에서 일반 scale로 변환됩니다
이렇게 변환된 값이 바로 오즈비입니다
오즈비는 "독립변수가 1단위 증가할 때 사건이 발생할 확률의 변화비율"을 의미합니다

예시로 설명하면:

weight의 coefficient가 0.0047이라면
np.exp(0.0047) = 1.0047입니다
해석: weight가 1단위 증가할 때마다 성공확률이 1.0047배(약 0.47% 증가)가 됩니다

이렇게 오즈비로 변환하면 계수의 실질적인 효과를 더 직관적으로 이해할 수 있습니다.

### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [ ]:
from statsmodels.formula.api import glm
import statsmodels.api as sm

#1) glm 모델 적합( 로지스틱 회귀를 위해 이항 분포 사용)
formula = "gender ~ age + length +diameter +height +weight"
model = glm(formula, data=train, family=sm.families.Binomial()).fit()       # familly는 로지스틱 회귀가 이항분포 사용한다고 하는것)

#2) 잔차이탈도 계산 (Deviance)
print(model.summary())
print(round(model.deviance,2))

#51.46

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 gender   No. Observations:                  210
Model:                            GLM   Df Residuals:                      204
Model Family:                Gaussian   Df Model:                            5
Link Function:               Identity   Scale:                         0.25226
Method:                          IRLS   Log-Likelihood:                -150.32
Date:                Thu, 26 Dec 2024   Deviance:                       51.461
Time:                        09:26:58   Pearson chi2:                     51.5
No. Iterations:                     3   Pseudo R-squ. (CS):            0.01841
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6647      0.161      4.122      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['familly']
  warnings.warn(msg, ValueWarning)


### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
#logit으로 공부해두는게 작업형 3에서는 좋음
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import logit

train = df.iloc[:210]
test = df.iloc[210:]


#1) 학습, test데이터를 사용해 예측 ( 0.5 미만: 0 , 0.5이상 : 1)
model = logit("gender ~ weight", data=train).fit()
target = test.pop("gender")
pred = model.predict(test) > 0.5
pred

#실제고 우리는 test 데이터에 gender 값이 있음 -> 그래서 예측해야하는 데이터인 test 데이터의 실제 gender 값을 pop으로 따로 뺴두고
#위에서 logit 모델을 통해 만들어진 예측모델을 통해 test 데이터의 gender 값을 예측해봄
#그래서 우리가 실제로 알고있는 실제 gender 값과 모델을 통해 예측한 gender 값을 accuaracy_score를 통해 얼마나 일치하는지 일치율을 보고
#오류가 얼마나 발생하는지 오류율을 계산 하는것 !!!

#2) 실제 값과 예측값을 사용하여 정확도 계선
acc = accuracy_score(target, pred)


#3) 오류율 계산
print(round(1-acc,3))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
0.478


In [ ]:
# 다른풀이
# 다른 풀이 (사이킷런)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

train = df.iloc[:210]
test = df.iloc[210:]

# 1) 학습 데이터와 테스트 데이터 분리
X_train = train[['weight']]
y_train = train['gender']
X_test = test[['weight']]
y_test = test['gender']

# 2) 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 3) 테스트 데이터를 사용해 예측
pred = model.predict(X_test)

# 4) 실제 값과 예측값을 사용하여 정확도 계산
acc = accuracy_score(y_test, pred)

# 5) 오류율 계산
print(round(1 - acc, 3))

### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")

#양의 상관관계인지 음의 상관관계인지 안나와있으면 절대값으로 생각하고 풀어라

In [ ]:
df.corr()
#0.434 이 정답 마이너스 생각말고 값이 가장 큰것!




,ERP,Feature1,Feature2,Feature3,CPU
ERP,1.000000,0.434442,0.076116,-0.059888,-0.201027
Feature1,0.434442,1.000000,0.076148,0.049980,-0.098571
Feature2,0.076116,0.076148,1.000000,0.126589,0.071456
Feature3,-0.059888,0.049980,0.126589,1.000000,-0.035247
CPU,-0.201027,-0.098571,0.071456,-0.035247,1.000000


### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
from statsmodels.formula.api import ols

#1) CPU가 100미만인 데이터 필터링
cond= df['CPU'] < 100
df= df[cond]

#2) 선형회귀 모델 생성: ERP를 종속변수로, 나머지 변수들을 독립 변수로 설정
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU', data = df).fit()


#3) 모델요약 정보 출력
print(model.summary())
print(model.rsquared)
print(model.rsquared_adj)

#결정계수 :  R- squared
#수정된 결정계수 : Adj. R-squared:
#0.226


                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Thu, 26 Dec 2024   Prob (F-statistic):           6.07e-05
Time:                        09:37:27   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [ ]:
#pvalues
#0.457
print(model.pvalues)


Intercept    0.000003
Feature1     0.000014
Feature2     0.456708
Feature3     0.298206
CPU          0.067968
dtype: float64
